# Day 11

## Part I

变体的康威生命游戏。按照标准写法来做，此处没有使用Numpy这样的库，因此没有应用矢量化函数进行运算，最终运行性能比较差。因此这里的代码主要为了清晰易懂而不是性能。

下面首先定义一个枚举类型，代表每个格子的可能状态，有三种空座位empty、被占座位occupied和地板floor：

In [1]:
from enum import Enum

CellState = Enum('CellState', ('empty', 'occupied', 'floor'))

然后是为了完成生命游戏的核心逻辑代码，此处如果应用Numpy数组和相应的矢量化函数运算的话，应该能提升不少性能，但是就失去了使用enum的代码清晰易懂特性：

In [12]:
from typing import List

def occupy_seats_count(cells: List[List[CellState]], row: int, col: int) -> int:
    '''
    计算一个单元格周围被占用
    '''
    drdc = ((dr, dc) for dr in range(-1, 2) for dc in range(-1, 2) if not (dr == 0 and dc == 0))
    count = 0
    for dr, dc in drdc:
        r = row + dr
        c = col + dc
        if r < 0 or c < 0 or r >= len(cells) or c >= len(cells[row]):
            continue
        if cells[r][c] == CellState.occupied:
            count += 1
    return count

def next_round(cells: List[List[CellState]]) -> List[CellState]:
    result = []
    for row, line in enumerate(cells):
        new_line = []
        for col, cell in enumerate(line):
            neighbor_occupied = occupy_seats_count(cells, row, col)
            if cell == CellState.empty and neighbor_occupied == 0:
                new_line.append(CellState.occupied)
            elif cell == CellState.occupied and neighbor_occupied >= 4:
                new_line.append(CellState.empty)
            else:
                new_line.append(cell)
        result.append(new_line)
    return result

In [13]:
def read_input(input_file: str) -> List[List[CellState]]:
    cells = []
    with open(input_file) as fn:
        line = fn.readline()
        while line:
            row = []
            for c in line.rstrip():
                if c == 'L':
                    row.append(CellState.empty)
                if c == '#':
                    row.append(CellState.occupied)
                if c == '.':
                    row.append(CellState.floor)
            cells.append(row)
            line = fn.readline()
    return cells

In [20]:
def count_all_occupied(cells: List[List[CellState]]) -> int:
    return sum(len([s for s in row if s == CellState.occupied]) for row in cells)

In [21]:
def part1_solution(cells: List[List[CellState]]) -> int:
    new_cells = next_round(cells)
    while True:
        if new_cells == cells:
            return count_all_occupied(cells)
        cells = new_cells
        new_cells = next_round(cells)

In [22]:
testcase = read_input('testcase1.txt')
part1_solution(testcase)

37

In [23]:
cells = read_input('input.txt')
part1_solution(cells)

2310

In [24]:
from typing import Generator

def drow_dcol_generator() -> Generator:
    i = 1
    while True:
        yield [(dr, dc) for dr in [-i, 0, i] for dc in [-i, 0, i] if not (dr == 0 and dc == 0)]
        i += 1

In [42]:
def one_direction(cells: List[List[CellState]], row: int, col: int, init_dr: int, init_dc: int) -> int:
    dr, dc = init_dr, init_dc
    while 0 <= row + dr < len(cells) and 0 <= col + dc < len(cells[row]):
        r, c = row + dr, col + dc
        if cells[r][c] == CellState.occupied:
            return 1
        if cells[r][c] == CellState.empty:
            return 0
        dr, dc = dr + init_dr, dc + init_dc
    return 0

def direction_occupied_count(cells: List[List[CellState]], row: int, col: int) -> int:
    count = 0
    drdc = ((dr, dc) for dr in range(-1, 2) for dc in range(-1, 2) if not (dr == 0 and dc == 0))
    for dr, dc in drdc:
        count += one_direction(cells, row, col, dr, dc)
    return count

In [43]:
def next_round(cells: List[List[CellState]], counter: callable, thresh: int) -> List[CellState]:
    result = []
    for row, line in enumerate(cells):
        new_line = []
        for col, cell in enumerate(line):
            neighbor_occupied = counter(cells, row, col)
            if cell == CellState.empty and neighbor_occupied == 0:
                new_line.append(CellState.occupied)
            elif cell == CellState.occupied and neighbor_occupied >= thresh:
                new_line.append(CellState.empty)
            else:
                new_line.append(cell)
        result.append(new_line)
    return result

In [44]:
def part2_solution(cells: List[List[CellState]]) -> int:
    new_cells = next_round(cells, direction_occupied_count, 5)
    while True:
        if new_cells == cells:
            return count_all_occupied(cells)
        cells = new_cells
        new_cells = next_round(cells, direction_occupied_count, 5)

In [45]:
part2_solution(testcase)

26

In [46]:
part2_solution(cells)

2074

In [47]:
def part1_solution(cells: List[List[CellState]]) -> int:
    new_cells = next_round(cells, occupy_seats_count, 4)
    while True:
        if new_cells == cells:
            return count_all_occupied(cells)
        cells = new_cells
        new_cells = next_round(cells, occupy_seats_count, 4)

In [48]:
part1_solution(cells)

2310